In [4]:
import json
import os
import webbrowser

from concurrent.futures import ThreadPoolExecutor

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

GH_TOKEN = os.environ.get("GHTOKEN")
GH_USER = 'dominikb1888'
GH_URL = "https://api.github.com/graphql"

with open('query.graphql', 'r') as query_file:
    query = query_file.read()

headers = { 'Content-Type': 'application/json', }
data = { 'query': query }
credentials = (GH_USER, GH_TOKEN)
# Send the POST request to the GraphQL endpoint
response = requests.post(GH_URL, json=data, headers=headers, auth=credentials)

# Check for errors
if response.status_code == 200:
    # If the request was successful, print the JSON response
    print(response.json())
else:
    # If there was an error, print the error message
    print(f"Request failed with status code {response.status_code}:")
    print(response.text)

{'data': {'organization': {'repositories': {'edges': [{'node': {'name': '07-08-complex-numbers-rodrigobivarazevedo', 'defaultBranchRef': {'target': {'history': {'edges': [{'node': {'id': 'C_kwDOKDc6BtoAKDExODM3MTI5Y2YxMDk5MWEzOTc0ODZmNjE3NDlhZjI3NWE1YzU4YjM', 'checkSuites': {'edges': [{'node': {'id': 'CS_kwDOKDc6Bs8AAAADdcTJWw', 'updatedAt': '2023-08-06T12:27:54Z', 'conclusion': None}}, {'node': {'id': 'CS_kwDOKDc6Bs8AAAADdcTKdA', 'updatedAt': '2023-08-06T12:28:12Z', 'conclusion': 'SUCCESS'}}]}, 'message': 'complex numbers', 'author': {'name': 'Rodrigo Azevedo', 'email': '52163635+rodrigobivarazevedo@users.noreply.github.com', 'date': '2023-08-06T12:27:31Z'}}}, {'node': {'id': 'C_kwDOKDc6BtoAKDAxNGY3MWI3NzIxNTg5YzUwZTQyZDllZDYyZGRlOWFkNjVkOGI5YjE', 'checkSuites': {'edges': [{'node': {'id': 'CS_kwDOKDc6Bs8AAAADc9ulqg', 'updatedAt': '2023-08-04T15:12:27Z', 'conclusion': None}}, {'node': {'id': 'CS_kwDOKDc6Bs8AAAADc9utig', 'updatedAt': '2023-08-04T15:12:47Z', 'conclusion': 'FAILURE'}}]}, 

In [103]:
df = pd.DataFrame(response.json())
df = pd.json_normalize(df['data'])
df = pd.json_normalize(df['repositories.edges']).T
df = pd.json_normalize(df[0])
df = df.explode('node.defaultBranchRef.target.history.edges')
df = df.join(pd.json_normalize(df['node.defaultBranchRef.target.history.edges']))
df = df.explode('node.checkSuites.edges')
df = df.join(pd.json_normalize(df['node.checkSuites.edges']), lsuffix='_right')
dff = df[['node.name','node.message','node.author.name', 'node.author.email', 'node.updatedAt', 'node.conclusion']]
dff[dff['node.conclusion'] == "FAILURE"]

,node.name,node.message,node.author.name,node.author.email,node.updatedAt,node.conclusion
9,r15-01-rest-api-lalaaadwmlq,GitHub Classroom Autograding Workflow,github-classroom[bot],66690702+github-classroom[bot]@users.noreply.g...,2023-08-04T15:12:47Z,FAILURE
9,r15-01-rest-api-lalaaadwmlq,GitHub Classroom Autograding Workflow,github-classroom[bot],66690702+github-classroom[bot]@users.noreply.g...,2023-08-04T15:12:47Z,FAILURE
9,r15-01-rest-api-lalaaadwmlq,GitHub Classroom Autograding Workflow,github-classroom[bot],66690702+github-classroom[bot]@users.noreply.g...,2023-08-04T15:12:47Z,FAILURE
9,r15-01-rest-api-lalaaadwmlq,GitHub Classroom Autograding Workflow,github-classroom[bot],66690702+github-classroom[bot]@users.noreply.g...,2023-08-04T15:12:47Z,FAILURE
9,r15-01-rest-api-lalaaadwmlq,GitHub Classroom Autograding Workflow,github-classroom[bot],66690702+github-classroom[bot]@users.noreply.g...,2023-08-04T15:12:47Z,FAILURE
...,...,...,...,...,...,...
35,r15-01-rest-api-axlgent007sam,Update GitHub Classroom Autograding,github-classroom[bot],66690702+github-classroom[bot]@users.noreply.g...,2023-08-07T09:55:41Z,FAILURE
35,r15-01-rest-api-axlgent007sam,Update GitHub Classroom Autograding,github-classroom[bot],66690702+github-classroom[bot]@users.noreply.g...,2023-08-07T09:55:41Z,FAILURE
35,r15-01-rest-api-axlgent007sam,Update GitHub Classroom Autograding,github-classroom[bot],66690702+github-classroom[bot]@users.noreply.g...,2023-08-07T09:55:41Z,FAILURE
35,r15-01-rest-api-axlgent007sam,Update GitHub Classroom Autograding,github-classroom[bot],66690702+github-classroom[bot]@users.noreply.g...,2023-08-07T09:55:41Z,FAILURE


In [75]:
pd.json_normalize(df, record_path='node.checkSuites.edges', meta=['node.name'])

TypeError: string indices must be integers

In [ ]:
df